In [56]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('train_data_with_features.csv')
df_test = pd.read_csv('test_data_with_features.csv')

# Movemos reason_end al final para el conjunto de train
df_train_cleaned = df_train[[c for c in df_train.columns if c != 'reason_end'] + ['reason_end']]
df_train_cleaned.head()

/var/folders/fx/50zkh6h57t37jktm101z3yvr0000gn/T/ipykernel_60727/1755709631.py:4: DtypeWarning: Columns (8,9,10,18,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('train_data_with_features.csv')


,ts,platform,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,...,obs_id,duration_normalized,explicit,popularity,track_number,available_markets_count,artist_uri,album_type,song_age,reason_end
0,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,The Eater Of Dreams,Nine Inch Nails,Hesitation Marks,spotify:track:1IPdwxRUbuNZiRpFN49RQC,NaN,NaN,...,1,0.093570,False,0.0,1.0,0.0,spotify:artist:0X380XXQSNBYuleKzav5UO,album,12.013689,fwdbtn
1,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,Copy Of A,Nine Inch Nails,Hesitation Marks,spotify:track:4BFKCEp4gwG3QHNlYodLMy,NaN,NaN,...,2,0.575761,False,0.0,2.0,0.0,spotify:artist:0X380XXQSNBYuleKzav5UO,album,12.013689,unknown
2,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,El extranjero,Bunbury,Singles,spotify:track:7ghPo5JVqcmwZQSpm6uyGG,NaN,NaN,...,3,0.475925,False,20.0,1.0,171.0,spotify:artist:4uqzzJg3ww5eH7IgGV7DMT,album,23.912389,trackdone
3,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,Luna,Bunbury,Singles,spotify:track:4g0Jk6sWxdu3FwwGDtoQPL,NaN,NaN,...,4,0.460320,False,27.0,2.0,171.0,spotify:artist:4uqzzJg3ww5eH7IgGV7DMT,album,23.912389,fwdbtn
4,2013-10-29 18:40:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,Feliz año,Bunbury,Singles,spotify:track:0GZzkyz7MN15llCyCBgPWh,NaN,NaN,...,5,0.430945,False,18.0,3.0,171.0,spotify:artist:4uqzzJg3ww5eH7IgGV7DMT,album,23.912389,fwdbtn


## Features

1. **Temporales**: year, month, day, time_of_day_sin, time_of_day_cos, day_of_week, is_weekend, is_morning, is_afternoon, is_evening, is_night

2. **User behavior**: track_order, time_since_last

3. **Track features**: duration_normalized, explicit, track_number, available_markets_count, album_type, is_song, is_episode

4. **Platform**: platform, conn_country, shuffle, offline, incognito_mode

### 0. Limpiamos columnas nulas

In [57]:
# Limpiamos las columnas nulas
df_train_cleaned = df_train_cleaned.drop(columns=['audiobook_title', 'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title', 'ip_addr', 
                                        'spotify_track_uri', 'spotify_episode_uri'])
df_test_cleaned = df_test.drop(columns=['audiobook_title', 'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title', 'ip_addr', 
                                        'spotify_track_uri', 'spotify_episode_uri'])

### 1. Features temporales

In [58]:
def temporal_features(df):
    df = df.copy()
    df['ts'] = pd.to_datetime(df['ts'])
    df['year'] = df['ts'].dt.year
    df['month'] = df['ts'].dt.month
    df['day'] = df['ts'].dt.day
    df['time_of_day'] = df['ts'].dt.time
    time_seconds = df['time_of_day'].apply(lambda t: t.hour * 3600 + t.minute * 60 + t.second)
    
    df.pop('ts')
    year = df.pop('year')
    month = df.pop('month')
    day = df.pop('day')
    day_of_week = pd.to_datetime({'year': year, 'month': month, 'day': day}).dt.day_name().astype('category')
    is_weekend = ((day_of_week == 'Saturday') | (day_of_week == 'Sunday')).astype(int)
    is_morning = ((time_seconds >= 21600) & (time_seconds < 43200)).astype(int)
    is_afternoon = ((time_seconds >= 43200) & (time_seconds < 64800)).astype(int)
    is_evening = ((time_seconds >= 64800) & (time_seconds < 79200)).astype(int)
    is_night = (((time_seconds >= 79200) | (time_seconds < 86400)) | (time_seconds < 21600)).astype(int)

    df.insert(0, 'year', year)
    df.insert(1, 'month', month)
    df.insert(2, 'day', day)
    df.insert(3, 'day_of_week', day_of_week)
    df.insert(4, 'is_weekend', is_weekend)
    df.insert(5, 'is_morning', is_morning)
    df.insert(6, 'is_afternoon', is_afternoon)
    df.insert(7, 'is_evening', is_evening)
    df.insert(8, 'is_night', is_night)

    
    # Cyclical encoding for time_of_day
    # Convert time_of_day to seconds since midnight
    seconds_in_day = 24 * 60 * 60
    df.pop('time_of_day')
    time_of_day_sin = np.sin(2 * np.pi * time_seconds / seconds_in_day)
    time_of_day_cos = np.cos(2 * np.pi * time_seconds / seconds_in_day)
    df.insert(9, 'time_of_day_sin', time_of_day_sin)
    df.insert(10, 'time_of_day_cos', time_of_day_cos)
    

    return df

df_train_cleaned = temporal_features(df_train_cleaned)
df_test_cleaned = temporal_features(df_test_cleaned)
df_train_cleaned.head()

,year,month,day,day_of_week,is_weekend,is_morning,is_afternoon,is_evening,is_night,time_of_day_sin,...,obs_id,duration_normalized,explicit,popularity,track_number,available_markets_count,artist_uri,album_type,song_age,reason_end
0,2013,10,29,Tuesday,0,0,0,1,1,-0.996195,...,1,0.093570,False,0.0,1.0,0.0,spotify:artist:0X380XXQSNBYuleKzav5UO,album,12.013689,fwdbtn
1,2013,10,29,Tuesday,0,0,0,1,1,-0.996195,...,2,0.575761,False,0.0,2.0,0.0,spotify:artist:0X380XXQSNBYuleKzav5UO,album,12.013689,unknown
2,2013,10,29,Tuesday,0,0,0,1,1,-0.996195,...,3,0.475925,False,20.0,1.0,171.0,spotify:artist:4uqzzJg3ww5eH7IgGV7DMT,album,23.912389,trackdone
3,2013,10,29,Tuesday,0,0,0,1,1,-0.996195,...,4,0.460320,False,27.0,2.0,171.0,spotify:artist:4uqzzJg3ww5eH7IgGV7DMT,album,23.912389,fwdbtn
4,2013,10,29,Tuesday,0,0,0,1,1,-0.984808,...,5,0.430945,False,18.0,3.0,171.0,spotify:artist:4uqzzJg3ww5eH7IgGV7DMT,album,23.912389,fwdbtn


In [59]:
def map_platform(platform_str):
    if pd.isnull(platform_str):
        return 'other'
    s = platform_str.lower()
    if 'windows' in s:
        return 'windows'
    elif 'android' in s:
        return 'android'
    elif 'ios' in s or 'iphone' in s or 'ipad' in s:
        return 'ios'
    elif 'linux' in s:
        return 'linux'
    elif 'osx' in s or 'mac' in s:
        return 'osx'
    elif 'cast' in s or 'chromecast' in s:
        return 'cast'
    elif 'webplayer' in s or 'web player' in s or 'web' in s:
        return 'webplayer'
    else:
        return 'other'

df_train_cleaned['platform'] = df_train_cleaned['platform'].apply(map_platform).astype('category')


In [60]:
# Cambiamos la columna reason_end a 1 (fwdbtn) o 0 (otro)
df_train_cleaned['reason_end'] = df_train_cleaned['reason_end'].apply(lambda x: 1 if x == 'fwdbtn' else 0)

In [61]:
categorical_columns = ['platform', 'conn_country', 'album_type', 'username']
for col in categorical_columns:
    df_train_cleaned[col] = df_train_cleaned[col].astype('category')

In [62]:
df_train_cleaned.to_csv('train_data_cleaned.csv', index=False)
df_test_cleaned.to_csv('test_data_cleaned.csv', index=False)


In [63]:
df_train_cleaned.columns

Index(['year', 'month', 'day', 'day_of_week', 'is_weekend', 'is_morning',
       'is_afternoon', 'is_evening', 'is_night', 'time_of_day_sin',
       'time_of_day_cos', 'platform', 'conn_country',
       'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'episode_name', 'episode_show_name',
       'shuffle', 'offline', 'offline_timestamp', 'incognito_mode', 'username',
       'obs_id', 'duration_normalized', 'explicit', 'popularity',
       'track_number', 'available_markets_count', 'artist_uri', 'album_type',
       'song_age', 'reason_end'],
      dtype='object')

In [64]:
# time_since_last_connection
# session_duration_approx
# user_avg_sessions_per_day

# user_order
# user_total_session
# session_progress
# is_first_session, is_last_session

In [65]:
# data leakage:
    # usar solo data available at prediction time